In [1]:
# importing libraries
from keras_preprocessing.image import ImageDataGenerator
import os
import keras
import numpy as np
import cv2
from sklearn.metrics import accuracy_score

In [2]:
# There are two classes of interests among the images in dataset
cls_of_int = ['Covid', 'Normal']

In [3]:
#setting the path variables for train, test and validation data
basePath = "Data"
trainPath = os.path.join(basePath,"train")
testPath = os.path.join(basePath,"test")
validPath = os.path.join(basePath,"valid")

trainCovid = os.path.join(trainPath,"Covid")
trainNormal = os.path.join(trainPath,"Normal")

testCovid = os.path.join(testPath,"Covid")
testNormal = os.path.join(testPath,"Normal")

validCovid = os.path.join(basePath,"valid","Covid")
validNormal = os.path.join(basePath,"valid","Normal")

In [4]:
# The data generator for test data, normalizing of the image is done
test_data_generator = ImageDataGenerator(rescale = 1./255)

In [5]:
# Read data from directory for testing data
test_generator = test_data_generator.flow_from_directory(
    directory= testPath,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=11,
    class_mode='binary',
    shuffle=False,
    seed=42
)

Found 2761 images belonging to 2 classes.


In [6]:
# combining the xception and inception model
xception_model = keras.models.load_model('imagegeneratormodelxc')
inception_model = keras.models.load_model('imagegeneratormodel')
models = [xception_model,inception_model]

In [8]:
# Make prediction using the ensemble technique
def ensemble_predictions(members):
  yhats = [model.evaluate(test_generator) for model in members]
  yhats = np.array(yhats)
  # sum across ensemble members
  summed = np.sum(yhats, axis=0)
  # argmax across classes
  result = np.argmax(summed, axis=1)
  return result

In [ ]:
# Method to call ensemble prediction against a list of file path
def readIm(filepaths):
    predictions = []
    for path in filepaths:
        img = cv2.imread(path)
        img = cv2.resize(img,(224,224))
        img = img[np.newaxis,...]
        img = img.astype(np.float32)        
        class_index = ensemble_predictions(models,img)[0]
        predictions.append(class_index)
    
    return predictions

In [9]:
# Calling prediction against test dataset
predictions = readIm(test_generator.filepaths)

# the real classes from test generator.
realclasses = test_generator.classes

#Calculating the accuracy score
accuracy = accuracy_score(realclasses,predictions)

251/251 [==============================] - 10s 34ms/step - loss: 0.5237 - accuracy: 0.7392


AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
# Printing accuracy score
print(accuracy)